In [101]:
import requests
import asyncio
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


# Cantidad de temporadas para analizar
seasons = range(2018,2024)

pg_stats = []

ad_stats = []

stats_by_year = []

col_labels = []



def get_nba_stats_by_seasons(seasons):
    
    
    
    for season in seasons:
    
        urlpg = 'https://www.basketball-reference.com/leagues/NBA_%i_per_game.html'%(season)
        urladv = 'https://www.basketball-reference.com/leagues/NBA_%i_advanced.html'%(season)
        
        df1 =  fetch_stats(urlpg)
        df2 =  fetch_stats(urladv)
        
        df1['season'] = season
        df2['season'] = season
        
        print(df1.head(), df2.head())
        
        if len(df1.index) == len(df2.index):
            pg_stats.append(df1)
            ad_stats.append(df2)
 
            

        else:
            print('frames n rows do not match')    
        
        
async def get_nba_stats_by_year(year) :
           
    urlpg = 'https://www.basketball-reference.com/leagues/NBA_%i_per_game.html'%(year)
    urladv = 'https://www.basketball-reference.com/leagues/NBA_%i_advanced.html'%(year)
    
    df1 = await fetch_stats(urlpg)
    df2 = await fetch_stats(urladv)
    
    
    if len(df1.index) == len(df2.index):
        print(df1.head())
        print(df2.head())
        dfmerged = ( pd.merge(left =df1, right= df2, how='outer'))
        stats_by_year.append(dfmerged)
        
       
    else:
        print('frames n rows do not match')
    

# URL of the webpage to scrape
url = 'https://www.basketball-reference.com/leagues/NBA_2023_advanced.html'

def fetch_stats(url):
    
    global col_labels


# Mandar un request a la url
    response =  requests.get(url)

    # Verificar el estado de la request (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the page using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find the table containing the data (assuming it's the first table on the page)
        table = soup.find('table')
        
        def custom_selector_rows(tag):

            return tag.name == 'td' and "DUMMY" != tag.get('data-stat')
        
        def custom_selector_cols(tag):

            return tag.name == 'th' and "DUMMY" != tag.get('data-stat') 
                
        
    
        # Extract the column headers from the table
        headers = []
        
        labels = []
        
        data_rows=[]
        
        # headers = [th.text for th in table.find_all(custom_selector_cols)]
        # print(headers)

        data = table.find_all('tr')
        
        headers.append([th.text for th in data[0].find_all(custom_selector_cols)])
        labels.append([th.get("aria-label") for th in data[0].find_all(custom_selector_cols)])
        headers = np.array(headers).flatten()
        labels = np.array (labels).flatten()
        
        if len(col_labels) == 0:
            col_labels = labels
        
        for row in data: 
        # Skip the first row (headers)

            data_rows.append([td.text for td in row.find_all(custom_selector_rows)])
        
        # print ( url, labels, headers, len(labels), len(headers))
        # print(data_rows)
        
        # Create a pandas DataFrame from the extracted data
    
        df = pd.DataFrame(data_rows[1:], columns=headers[1:])
        return df
        
        
        # Print the DataFrame (you can also save it to a CSV file or perform further analysis)

    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}") 
    
    

In [102]:
get_nba_stats_by_seasons(seasons)

          Player Pos Age   Tm   G  GS    MP   FG  FGA   FG%  ...  ORB  DRB  \
0   Álex Abrines  SG  24  OKC  75   8  15.1  1.5  3.9  .395  ...  0.3  1.2   
1     Quincy Acy  PF  27  BRK  70   8  19.4  1.9  5.2  .356  ...  0.6  3.1   
2   Steven Adams   C  24  OKC  76  76  32.7  5.9  9.4  .629  ...  5.1  4.0   
3    Bam Adebayo   C  20  MIA  69  19  19.8  2.5  4.9  .512  ...  1.7  3.8   
4  Arron Afflalo  SG  32  ORL  53   3  12.9  1.2  3.1  .401  ...  0.1  1.2   

   TRB  AST  STL  BLK  TOV   PF   PTS season  
0  1.5  0.4  0.5  0.1  0.3  1.7   4.7   2018  
1  3.7  0.8  0.5  0.4  0.9  2.1   5.9   2018  
2  9.0  1.2  1.2  1.0  1.7  2.8  13.9   2018  
3  5.5  1.5  0.5  0.6  1.0  2.0   6.9   2018  
4  1.2  0.6  0.1  0.2  0.4  1.1   3.4   2018  

[5 rows x 30 columns]           Player Pos Age   Tm   G    MP   PER   TS%  3PAr   FTr  ...  USG%  \
0   Álex Abrines  SG  24  OKC  75  1134   9.0  .567  .759  .158  ...  12.7   
1     Quincy Acy  PF  27  BRK  70  1359   8.2  .525  .800  .164  ...  

In [103]:
print(len(pg_stats), len(ad_stats))

df = []

for x, y in zip(pg_stats,ad_stats):
    cols = y.columns.difference(x.columns)
    dfmerged = x.join(y[cols])
    df.append(dfmerged)
    print(dfmerged)
    


6 6
            Player Pos Age   Tm   G  GS    MP   FG  FGA   FG%  ...   OWS  \
0     Álex Abrines  SG  24  OKC  75   8  15.1  1.5  3.9  .395  ...   1.3   
1       Quincy Acy  PF  27  BRK  70   8  19.4  1.9  5.2  .356  ...  -0.1   
2     Steven Adams   C  24  OKC  76  76  32.7  5.9  9.4  .629  ...   6.7   
3      Bam Adebayo   C  20  MIA  69  19  19.8  2.5  4.9  .512  ...   2.3   
4    Arron Afflalo  SG  32  ORL  53   3  12.9  1.2  3.1  .401  ...  -0.1   
..             ...  ..  ..  ...  ..  ..   ...  ...  ...   ...  ...   ...   
685   Tyler Zeller   C  28  BRK  42  33  16.7  3.0  5.5  .546  ...   1.0   
686   Tyler Zeller   C  28  MIL  24   1  16.9  2.6  4.4  .590  ...   1.1   
687    Paul Zipser  SF  23  CHI  54  12  15.3  1.5  4.3  .346  ...  -1.1   
688     Ante Žižić   C  21  CLE  32   2   6.7  1.5  2.1  .731  ...   0.9   
689    Ivica Zubac   C  20  LAL  43   0   9.5  1.4  2.8  .500  ...   0.5   

      PER STL%  TOV%  TRB%   TS%  USG%  VORP    WS   WS/48  
0     9.0  1.7   7.4  

In [104]:
df_nba = pd.concat(df)

csv_nba = df_nba.to_csv('C:/Users/Rodolfo/Downloads/nba_stats.csv')


print(df_nba.describe(include='object'))

    

                 Player   Pos   Age    Tm     G    GS    MP    FG   FGA   FG%  \
count              4219  4219  4219  4219  4219  4219  4219  4219  4219  4219   
unique             1086    16    25    31    83    84   364   113   221   466   
top     James Ennis III    SG    23   TOT     1     0   3.0   1.0   1.0  .500   
freq                 14   993   443   451   101  1205    35   128    85   110   

        ...   OWS   PER  STL%  TOV%  TRB%   TS%  USG%  VORP    WS WS/48  
count   ...  4219  4219  4219  4219  4219  4219  4219  4219  4219  4219  
unique  ...   125   408    83   307   273   489   325    90   145   535  
top     ...   0.0  12.1   0.0   0.0   0.0  .000  15.7   0.0   0.0  .062  
freq    ...   523    44   307   192    83    60    47   793   436    39  

[4 rows x 49 columns]


La idea es generar un modelo que me permita dado un dataset de jugadores rankearlos para un Draft de fantasy que tiene un número de condiciones especificas, por ejemplo un Fantasy de 10 categorías con 10 personas y un draft tipo snake.etc Y me diga con esas restricciones que jugador tiene un valor de sustitución mayor.

Las variables más importantes son la posición, los minutos jugados(ayuda a eliminar data que puede ser ruido), las categorías básicas de un fantasy de basketball que es fg%, 3pts, pts, rebotes, robos, bloqueos, faltas, pérdidas de balón. Se podría también explorar si hay otras variables que se correlacionan de manera importante (pueden predecir) con un rendimiento fuerte en las categorias básicas. 

Sephora Dataset

In [89]:
df_sephora = pd.read_csv('C:/Users/Rodolfo/Downloads/sephoradataset/product_info.csv')
print(df_sephora.shape, df_sephora.describe)


(8494, 27) <bound method NDFrame.describe of      product_id                                       product_name  brand_id  \
0       P473671                            Fragrance Discovery Set      6342   
1       P473668                            La Habana Eau de Parfum      6342   
2       P473662                          Rainbow Bar Eau de Parfum      6342   
3       P473660                               Kasbah Eau de Parfum      6342   
4       P473658                          Purple Haze Eau de Parfum      6342   
...         ...                                                ...       ...   
8489    P467659                   Couture Clutch Eyeshadow Palette      1070   
8490    P500874                              L'Homme Eau de Parfum      1070   
8491    P504428                   Mon Paris Eau de Parfum Gift Set      1070   
8492    P504448                           Y Eau de Parfum Gift Set      1070   
8493    P505461  Candy Glaze Lip Gloss Stick Duo with Hyaluroni...      107

Las variables lovescount(debe ser como un like) y los reviews ayudan  crear un punto de comparación donde las variables cualitativas permiten  hacer modelo donde que caracteristicas de un producto van a tener un buen review  y merece la pena dedicar más dinero a ese tipo de productos

Glassdor Job Posting Reviews Dataset

In [98]:
df_glassdoor = pd.read_csv('C:/Users/Rodolfo/Downloads/glassdordataset/glassdoor_reviews.csv')

print(df_glassdoor.describe(include='all'))

          firm date_review            job_title           current  \
count   838566      838566               838566            838566   
unique     428        4813                62275                29   
top        IBM  2021-01-13   Anonymous Employee  Current Employee   
freq     60436        1761               162649            209599   
mean       NaN         NaN                  NaN               NaN   
std        NaN         NaN                  NaN               NaN   
min        NaN         NaN                  NaN               NaN   
25%        NaN         NaN                  NaN               NaN   
50%        NaN         NaN                  NaN               NaN   
75%        NaN         NaN                  NaN               NaN   
max        NaN         NaN                  NaN               NaN   

                        location  overall_rating  work_life_balance  \
count                     541223   838566.000000      688672.000000   
unique                     14

Al igual que en el dataset tenemos una cantidad de variables continuas que ponderan la calidad de un trabajo acompañado de una serie de varaibles cualitativas. Se pueden aprovechar también para aproximar los valores de esas varaibles que predicen que un trabajo posteado baya a ser considerado como un buen puesto de trabajo.